In [ ]:
!pip install -q langchain transformers accelerate bitsandbytes langchain ctransformers sentence-transformers faiss-cpu

Load the libraries

In [ ]:
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline

import sys
import textwrap

from langchain.chains import LLMChain
from langchain import HuggingFacePipeline
from langchain import PromptTemplate
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain

Downloading the model and tokenizer from HuggingFace

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf")

In [ ]:
model = AutoModelForCausalLM.from_pretrained("NousResearch/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             load_in_4bit=True,
                                             bnb_4bit_quant_type="nf4",
                                             bnb_4bit_compute_dtype=torch.float16)


Define Transformers Pipeline which will be fed into Langchain


In [ ]:
pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.float16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )